In [1]:
import pandas as pd
import speech_recognition as sr
from transformers import pipeline
import Levenshtein

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [7]:
model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Use it in the pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cpu


In [21]:
def load_ipa_dataset(file_path):
    df = pd.read_csv('phoneticDictionary.csv')
    df['word'] = df['word'].astype(str)
    df['phon'] = df['phon'].astype(str)
    df['syl'] = df['syl'].astype(int)
    df['start'] = df['start'].astype(str)
    df['end'] = df['end'].astype(str)
    return df

In [23]:
def generate_valid_word(difficulty, df):
    prompt = {
        "easy": "Simple words: ",
        "medium": "Moderate words: ",
        "hard": "Complex words: "
    }[difficulty]
    
    while True:
        generated_text = generator(prompt, max_length=10, num_return_sequences=1)
        candidate_word = generated_text[0]['generated_text'].split()[0].strip().lower()
        
        if candidate_word in df['word'].str.lower().values:
            return candidate_word

In [25]:
def record_user_audio(filename="user_audio.wav"):
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Please pronounce the magical word...")
        recognizer.adjust_for_ambient_noise(source)
        audio_data = recognizer.listen(source)
        print("Audio recorded.")

        with open(filename, "wb") as f:
            f.write(audio_data.get_wav_data())
            
    return audio_data

In [27]:
def analyze_pronunciation(user_audio, target_word, df, tolerance=0.6, poor_match_threshold=0.4):
    recognizer = sr.Recognizer()

    try:
        user_pronunciation = recognizer.recognize_google(user_audio)
        print(f"User Pronunciation: {user_pronunciation}")

        target_row = df[df['word'].str.lower() == target_word.lower()]
        if not target_row.empty:
            correct_pronunciation = target_row['word'].values[0]
            ipa_pronunciation = target_row['phon'].values[0]
            print(f"Correct Pronunciation (IPA): {ipa_pronunciation}")

            user_pronunciation = user_pronunciation.replace(" ", "").lower()
            correct_pronunciation = correct_pronunciation.replace(" ", "").lower()

            edit_distance = Levenshtein.distance(user_pronunciation, correct_pronunciation)

            max_len = max(len(user_pronunciation), len(correct_pronunciation))
            similarity = 1 - (edit_distance / max_len)

            print(f"Similarity Score: {similarity}")

            if similarity < poor_match_threshold:
                print("Your pronunciation is significantly different.")
                return 0 
            elif similarity < tolerance:
                print("Your pronunciation is slightly different, but acceptable.")
                return similarity  
            return similarity
        
        else:
            print("Target word not found in dataset")
            return 0 
            
    except sr.UnknownValueError:
        print("Could not understand audio")
        return 0
    except sr.RequestError:
        print("Request failed")
        return 0             

In [29]:
def provide_feedback(similarity):
    if similarity is None:
        print("Pronunciation analysis could not be completed.")
        return
    
    if similarity > 0.9:
        print("Excellent! Your pronunciation is almost perfect.")
    elif similarity > 0.75:
        print("Good job! Your pronunciation is quite accurate.")
    elif similarity >= 0.5:
        print("Not bad! But there's some room for improvement.")
    else:
        print("It seems like your pronunciation is quite different. Try again.")

In [31]:
def spell_generation_game(file_path):
    df = load_ipa_dataset(file_path)
    difficulty = "medium"  # Start with medium difficulty
    
    while True:
        magical_word = generate_valid_word(difficulty, df)
        print(f"Your magical word is: {magical_word}")
        
        matching_rows = df[df['word'].str.lower() == magical_word.lower()]
        if not matching_rows.empty:
            print(f"The word {magical_word} exists in the dataset.")
            print("Here are some details:")
            print(matching_rows[['word', 'phon', 'syl', 'start', 'end']].head())
            
            user_audio = record_user_audio()
            similarity = analyze_pronunciation(user_audio, magical_word, df)
            provide_feedback(similarity)

            if similarity > 0.9:
                difficulty = "hard"
            elif similarity > 0.5:
                difficulty = "medium"
            else:
                difficulty = "easy"
        else:
            print(f"The word {magical_word} does not exist in the dataset.")
        
        play_again = input("Would you like to try another word? (yes/no): ").strip().lower()
        if play_again != "yes":
            print("Thanks for playing! Goodbye.")
            break

In [35]:
spell_generation_game('phoneticDictionary.csv')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Your magical word is: moderate
The word moderate exists in the dataset.
Here are some details:
           word       phon  syl start end
80214  moderate  ˈmɑdɝˌeɪt    3     ɑ  eɪ
80215  moderate    ˈmɑdɝət    2     ɑ  ɝə
Please pronounce the magical word...
Audio recorded.
User Pronunciation: moderate
Correct Pronunciation (IPA): ˈmɑdɝˌeɪt
Similarity Score: 1.0
Excellent! Your pronunciation is almost perfect.


Would you like to try another word? (yes/no):  yes


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Your magical word is: complex
The word complex exists in the dataset.
Here are some details:
          word       phon  syl start end
23845  complex  ˈkɑmpɫɛks    2     ɑ   ɛ
23846  complex  kəmˈpɫɛks    2     ə   ɛ
Please pronounce the magical word...
Audio recorded.
User Pronunciation: complex
Correct Pronunciation (IPA): ˈkɑmpɫɛks
Similarity Score: 1.0
Excellent! Your pronunciation is almost perfect.


...

Would you like to try another word? (yes/no):  yes


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Your magical word is: simple
The word simple exists in the dataset.
Here are some details:
          word     phon  syl start end
111289  simple  ˈsɪmpəɫ    2     ɪ   ə
Please pronounce the magical word...
Audio recorded.
Could not understand audio
It seems like your pronunciation is quite different. Try again.


Would you like to try another word? (yes/no):  no


Thanks for playing! Goodbye.


In [37]:
import pandas as pd
import speech_recognition as sr
from transformers import pipeline
import Levenshtein

In [39]:
model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Use it in the pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cpu


In [53]:
def load_ipa_dataset(file_path):
    df = pd.read_csv('phoneticdict_with_labels.csv')
    df['word'] = df['word'].astype(str)
    df['phon'] = df['phon'].astype(str)
    df['syl'] = df['syl'].astype(int)
    df['start'] = df['start'].astype(str)
    df['end'] = df['end'].astype(str)
    df['level'] = df['level'].astype(str)  # Assuming 'level' is a column in your dataset
    return df

In [163]:
def generate_valid_word(difficulty, df, prev_word, prev_similarity):
    prompt = f"Generate a {difficulty} word: "
    
    while True:
        generated_text = generator(prompt, max_length=10, num_return_sequences=1)
        candidate_word = generated_text[0]['generated_text'].split()[0].strip().lower()
        
        if candidate_word in df['word'].str.lower().values:
            return candidate_word

In [165]:
def record_user_audio(filename="user_audio.wav"):
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Please pronounce the magical word...")
        recognizer.adjust_for_ambient_noise(source)
        audio_data = recognizer.listen(source)
        print("Audio recorded.")

        with open(filename, "wb") as f:
            f.write(audio_data.get_wav_data())
            
    return audio_data

In [167]:
def analyze_pronunciation(user_audio, target_word, df, tolerance=0.6, poor_match_threshold=0.4):
    recognizer = sr.Recognizer()

    try:
        user_pronunciation = recognizer.recognize_google(user_audio)
        print(f"User Pronunciation: {user_pronunciation}")

        target_row = df[df['word'].str.lower() == target_word.lower()]
        if not target_row.empty:
            correct_pronunciation = target_row['word'].values[0]
            ipa_pronunciation = target_row['phon'].values[0]
            print(f"Correct Pronunciation (IPA): {ipa_pronunciation}")

            user_pronunciation = user_pronunciation.replace(" ", "").lower()
            correct_pronunciation = correct_pronunciation.replace(" ", "").lower()

            edit_distance = Levenshtein.distance(user_pronunciation, correct_pronunciation)

            max_len = max(len(user_pronunciation), len(correct_pronunciation))
            similarity = 1 - (edit_distance / max_len)

            print(f"Similarity Score: {similarity}")

            if similarity < poor_match_threshold:
                print("Your pronunciation is significantly different.")
                return 0 
            elif similarity < tolerance:
                print("Your pronunciation is slightly different, but acceptable.")
                return similarity  
            return similarity
        
        else:
            print("Target word not found in dataset")
            return 0 
            
    except sr.UnknownValueError:
        print("Could not understand audio")
        return 0
    except sr.RequestError:
        print("Request failed")
        return 0 


In [169]:

def provide_feedback(similarity):
    if similarity is None:
        print("Pronunciation analysis could not be completed.")
        return
    
    if similarity > 0.9:
        print("Excellent! Your pronunciation is almost perfect.")
    elif similarity > 0.75:
        print("Good job! Your pronunciation is quite accurate.")
    elif similarity >= 0.5:
        print("Not bad! But there's some room for improvement.")
    else:
        print("It seems like your pronunciation is quite different. Try again.")


In [171]:
def spell_generation_game(file_path):
    df = load_ipa_dataset('phoneticdict_with_labels.csv')
    difficulty = "simple"  # Start with simple difficulty
    correct_streak = 0
    incorrect_streak = 0
    prev_word = None
    prev_similarity = None
    
    while True:
        magical_word = generate_valid_word(difficulty, df, prev_word, prev_similarity)
        print(f"Your magical word is: {magical_word}")
        
        matching_rows = df[df['word'].str.lower() == magical_word.lower()]
        if not matching_rows.empty:
            print(f"The word {magical_word} exists in the dataset.")
            print("Here are some details:")
            print(matching_rows[['word', 'phon', 'syl', 'start', 'end', 'level']].head())
            
            user_audio = record_user_audio()
            similarity = analyze_pronunciation(user_audio, magical_word, df)

            provide_feedback(similarity)

            # Track streaks
            if similarity > 0.75:
                correct_streak += 1
                incorrect_streak = 0
            else:
                incorrect_streak += 1
                correct_streak = 0
            
            # Adjust difficulty based on performance streaks
            if correct_streak >= 2:
                if difficulty == "simple":
                    difficulty = "moderate"
                elif difficulty == "moderate":
                    difficulty = "complex"
                correct_streak = 0  # Reset streak after difficulty change
            elif incorrect_streak >= 2:
                if difficulty == "complex":
                    difficulty = "moderate"
                elif difficulty == "moderate":
                    difficulty = "simple"
                incorrect_streak = 0  # Reset streak after difficulty change

            # Store previous word and similarity for the next round
            prev_word = magical_word
            prev_similarity = similarity

        else:
            print(f"The word {magical_word} does not exist in the dataset.")
        
        play_again = input("Would you like to try another word? (yes/no): ").strip().lower()
        if play_again != "yes":
            print("Thanks for playing! Goodbye.")
            break

In [173]:
spell_generation_game('phoneticdict_with_labels.csv')

TypeError: generate_valid_word() takes 2 positional arguments but 4 were given

In [111]:
def spell_generation_game1(file_path):
    df = load_ipa_dataset('phoneticdict_with_labels.csv')
    difficulty = "simple"
    correct_streak = 0
    incorrect_streak = 0
    prev_word = None
    prev_similarity = None
    
    while True:
        # Generate a valid word
        magical_word = generate_valid_word(difficulty, df, prev_word, prev_similarity)
        print(f"Generated word: {magical_word}")
        
        # Check if the generated word exists in the dataset
        matching_rows = df[df['word'].str.lower() == magical_word.lower()]
        print(f"Matching rows in dataset: {matching_rows[['word', 'phon', 'syl', 'start', 'end', 'level']].head()}")  # Debug output
        
        if not matching_rows.empty:
            print(f"The word {magical_word} exists in the dataset.")
            print("Here are some details:")
            print(matching_rows[['word', 'phon', 'syl', 'start', 'end', 'level']].head())
            
            user_audio = record_user_audio()
            similarity = analyze_pronunciation(user_audio, magical_word, df)
            print(f"User pronunciation similarity: {similarity}")  # Debug output

            provide_feedback(similarity)

            # Track streaks
            if similarity > 0.75:
                correct_streak += 1
                incorrect_streak = 0
            else:
                incorrect_streak += 1
                correct_streak = 0
            
            # Adjust difficulty based on performance streaks
            if correct_streak >= 2:
                if difficulty == "simple":
                    difficulty = "moderate"
                elif difficulty == "moderate":
                    difficulty = "complex"
                correct_streak = 0
            elif incorrect_streak >= 2:
                if difficulty == "complex":
                    difficulty = "moderate"
                elif difficulty == "moderate":
                    difficulty = "simple"
                incorrect_streak = 0

            print(f"Difficulty is now: {difficulty}")  # Debug output
            prev_word = magical_word
            prev_similarity = similarity
        else:
            print(f"The word {magical_word} does not exist in the dataset.")
        
        play_again = input("Would you like to try another word? (yes/no): ").strip().lower()
        if play_again != "yes":
            print("Thanks for playing! Goodbye.")
            break


In [113]:
spell_generation_game1('phoneticdict_with_labels.csv')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Word: generate
Generated word: generate
Matching rows in dataset:            word        phon  syl start end    level
47221  generate  ˈdʒɛnɝˌeɪt    3     ɛ  eɪ  complex
The word generate exists in the dataset.
Here are some details:
           word        phon  syl start end    level
47221  generate  ˈdʒɛnɝˌeɪt    3     ɛ  eɪ  complex
Please pronounce the magical word...
Audio recorded.
User Pronunciation: generate
Correct Pronunciation (IPA): ˈdʒɛnɝˌeɪt
Similarity Score: 1.0
User pronunciation similarity: 1.0
Excellent! Your pronunciation is almost perfect.
Difficulty is now: simple


Would you like to try another word? (yes/no):  yes


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Word: generate
Generated word: generate
Matching rows in dataset:            word        phon  syl start end    level
47221  generate  ˈdʒɛnɝˌeɪt    3     ɛ  eɪ  complex
The word generate exists in the dataset.
Here are some details:
           word        phon  syl start end    level
47221  generate  ˈdʒɛnɝˌeɪt    3     ɛ  eɪ  complex
Please pronounce the magical word...
Audio recorded.
User Pronunciation: generate
Correct Pronunciation (IPA): ˈdʒɛnɝˌeɪt
Similarity Score: 1.0
User pronunciation similarity: 1.0
Excellent! Your pronunciation is almost perfect.
Difficulty is now: moderate


Would you like to try another word? (yes/no):  no


Thanks for playing! Goodbye.
